In [1]:
!python -V

Python 3.12.9


In [7]:
import numpy as np
import pandas as pd

In [8]:
import pickle

In [9]:
import seaborn as sns
import matplotlib.pyplot as plt

In [10]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [6]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/workspaces/mlops-zoomcamp-2025-practice/02-experiment-tracking/mlruns/1', creation_time=1748283763739, experiment_id='1', last_update_time=1748283763739, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [11]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [12]:
# df_train = read_dataframe('./data/green_tripdata_2021-01.csv')
# df_val = read_dataframe('./data/green_tripdata_2021-02.csv')

df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')

In [13]:
len(df_train), len(df_val)

(73908, 61921)

In [14]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [15]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [16]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [13]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

# mean_squared_error(y_val, y_pred, squared=False)

np.sqrt(mean_squared_error(y_val, y_pred))

np.float64(7.758715209663881)

In [14]:
with open('../models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [16]:
with mlflow.start_run():

    mlflow.set_tag("developer", "ranga")

    mlflow.log_param("train-data-path", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet")

    alpha = 0.01
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="../models/lin_reg.bin", artifact_path="models_pickle")

In [19]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [20]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [23]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=100,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [24]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    # 'objective': 'reg:linear',
    'objective': 'reg:squarederror',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[0]	validation-rmse:11.26796                                                                                                                                      
[1]	validation-rmse:10.47572                                                                                                                                      
[2]	validation-rmse:9.81321                                                                                                                                       
[3]	validation-rmse:9.26500                                                                                                                                       
[4]	validation-rmse:8.81247                                                                                                                                       
[5]	validation-rmse:8.44375                                                                                                                                       
[6]	validation-rmse:8.

KeyboardInterrupt: 

In [25]:
mlflow.xgboost.autolog(disable=True)

In [27]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=100,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    mlflow.log_metric("rmse", rmse)

    with open("../models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("../models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

/opt/conda/envs/mlops/lib/python3.12/site-packages/xgboost/callback.py:386: UserWarning: [21:50:10] WARNING: /workspace/src/objective/regression_obj.cu:250: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()


[0]	validation-rmse:11.44482
[1]	validation-rmse:10.77202
[2]	validation-rmse:10.18363
[3]	validation-rmse:9.67396
[4]	validation-rmse:9.23166
[5]	validation-rmse:8.84808
[6]	validation-rmse:8.51883
[7]	validation-rmse:8.23597
[8]	validation-rmse:7.99320
[9]	validation-rmse:7.78709
[10]	validation-rmse:7.61022
[11]	validation-rmse:7.45952
[12]	validation-rmse:7.33049
[13]	validation-rmse:7.22098
[14]	validation-rmse:7.12713
[15]	validation-rmse:7.04752
[16]	validation-rmse:6.98005
[17]	validation-rmse:6.92232
[18]	validation-rmse:6.87112
[19]	validation-rmse:6.82740
[20]	validation-rmse:6.78995
[21]	validation-rmse:6.75792
[22]	validation-rmse:6.72994
[23]	validation-rmse:6.70547
[24]	validation-rmse:6.68390
[25]	validation-rmse:6.66421
[26]	validation-rmse:6.64806
[27]	validation-rmse:6.63280
[28]	validation-rmse:6.61924
[29]	validation-rmse:6.60773
[30]	validation-rmse:6.59777
[31]	validation-rmse:6.58875
[32]	validation-rmse:6.58107
[33]	validation-rmse:6.57217
[34]	validation-rmse:

/opt/conda/envs/mlops/lib/python3.12/site-packages/mlflow/xgboost/__init__.py:168: UserWarning: [21:50:56] WARNING: /workspace/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/05/26 21:51:03 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

## Predict using logged model

In [3]:
import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

logged_model = 'runs:/222ab993adbf4405aaa6aef355c10875/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)


INFO:alembic.runtime.migration:Context impl SQLiteImpl.
INFO:alembic.runtime.migration:Will assume non-transactional DDL.


In [ ]:
loaded_model

In [4]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: 222ab993adbf4405aaa6aef355c10875

In [5]:
# Load model as xgboost model

xgboost_model = mlflow.xgboost.load_model(logged_model)

In [6]:
xgboost_model

In [22]:
# Make predictions

y_pred = xgboost_model.predict(valid)

In [23]:
y_pred

array([15.707287 ,  7.1738143, 18.640547 , ..., 13.417416 ,  6.7913156,
        8.298626 ], shape=(61921,), dtype=float32)

## Using preprocessor + lodded model

In [28]:
import mlflow
import mlflow.xgboost
import pickle
import os

# Set up MLflow tracking
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

# Define the run ID
run_id = "222ab993adbf4405aaa6aef355c10875"

# Load model as xgboost model
logged_model = f'runs:/{run_id}/models_mlflow'
xgboost_model = mlflow.xgboost.load_model(logged_model)

# Load preprocessor from artifact
client = mlflow.tracking.MlflowClient()
preprocessor_path = client.download_artifacts(run_id, "preprocessor/preprocessor.b")

# Load the pickle file
with open(preprocessor_path, "rb") as f_in:
    preprocessor = pickle.load(f_in)

print("Model and preprocessor loaded successfully!")
print(f"XGBoost model type: {type(xgboost_model)}")
print(f"Preprocessor type: {type(preprocessor)}")

# Example usage function
def predict_with_preprocessing(raw_data):
    """
    Make predictions using both preprocessor and model
    
    Args:
        raw_data: Raw input data (pandas DataFrame or dict)
    
    Returns:
        predictions: Model predictions
    """
    # Apply preprocessing (assuming dv is a DictVectorizer or similar)
    if hasattr(preprocessor, 'transform'):
        # For sklearn-style transformers
        processed_data = preprocessor.transform(raw_data)
    else:
        # For custom preprocessors
        processed_data = preprocessor.transform(raw_data)
    
    # Convert to DMatrix for XGBoost prediction
    import xgboost as xgb
    dmatrix = xgb.DMatrix(processed_data)
    
    # Make predictions
    predictions = xgboost_model.predict(dmatrix)
    
    return predictions

# Alternative: If you want to clean up the downloaded file after loading
# os.remove(preprocessor_path)

Model and preprocessor loaded successfully!
XGBoost model type: <class 'xgboost.core.Booster'>
Preprocessor type: <class 'sklearn.feature_extraction._dict_vectorizer.DictVectorizer'>


In [30]:
y_pred = predict_with_preprocessing(val_dicts)

In [31]:
y_pred[:10]

array([15.707287 ,  7.1738143, 18.640547 , 24.26097  ,  9.868029 ,
       17.068527 , 13.516277 ,  9.015495 ,  8.919762 , 19.05189  ],
      dtype=float32)